# tensorflow pistachio


In [1]:
import tensorflow as tf
print(tf.__version__)

2024-05-03 15:33:54.530098: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.16.1


## arff to csv





In [2]:
import pandas as pd 
from scipy.io import arff
import os 
label_mapping = {'Kirmizi_Pistachio': 0, 'Siit_Pistachio': 1}

def load_arff_file(input_arff: str) -> pd.DataFrame:
    """convert arff file to parquet"""
    if not os.path.exists(input_arff):
        raise ValueError(f"input file '{input_arff}' does not exist")
    print(f'loading arff file {input_arff}')
    data, meta = arff.loadarff(input_arff)
    print(f"arff metadata: {meta}")
    df = pd.DataFrame(data)
    df['Class'] = df['Class'].astype(str).map(label_mapping)
    
    return df
##################

arff_filename = './data/Pistachio_16_Features_Dataset.arff'
csv_filename = './data/pistachio_16.csv'
if not os.path.exists(csv_filename):
    df = load_arff_file(arff_filename)
    df.head()
    df.to_csv(csv_filename, index=False, header=True)
    print(f'wrote file to {csv_filename}')
else:
    print(f'{csv_filename} exists')


./data/pistachio_16.csv exists


## dataset


In [3]:
import numpy as np
from typing import List
# want a stratified split here
def split_csv_data(infilename: str, filenames: List[str], fractions: List[float]):
    df = pd.read_csv(infilename, header=0)
    df = df.sample(frac=1.0, random_state=34)
    columns = df.columns
    print(f'total_records = {len(df)}')
    if len(filenames) != len(fractions):
        raise ValueError('list of filenames must be of same length as split fractions')
    renorm = sum(fractions)
    lower_bound = 0
    df['split_var'] = np.random.uniform(size=len(df))

    for filename, frac in zip(filenames, fractions):
        upper_bound = lower_bound + frac/renorm
        this_data = df.loc[(df.split_var >= lower_bound) & (df.split_var < upper_bound) ][columns]
        this_data.to_csv(filename, index=False, header=True)
        print(f'wrote {len(this_data)} records to {filename}')
        lower_bound = upper_bound
#################################
    
train_filename = './data/pistachio_train.csv'
valid_filename = './data/pistachio_valid.csv'
test_filename = './data/pistachio_test.csv'
filenames = [train_filename, valid_filename, test_filename]
fractions = [0.75, 0.10, 0.15]

if not (os.path.exists(train_filename) and os.path.exists(test_filename) and os.path.exists(valid_filename)):
    split_csv_data(csv_filename, filenames, fractions )
else:
    print(f'{filenames} exist')






['./data/pistachio_train.csv', './data/pistachio_valid.csv', './data/pistachio_test.csv'] exist


In [11]:
def map_func(features, labels):
    return tf.transpose(tf.stack([features[k] for k in features])), tf.reshape(labels,[-1,1])

# use dataset.map to concatenate feature dictionary into tensor
pistachio_train_batches = tf.data.experimental.make_csv_dataset(
    train_filename, batch_size=4,
    num_epochs=1,
    label_name="Class").map(map_func)
pistachio_test_data = tf.data.experimental.make_csv_dataset(
    test_filename,
    batch_size=4,
    num_epochs=1,
    label_name="Class").map(map_func)
validation_data = tf.data.experimental.make_csv_dataset(
    valid_filename, batch_size=4,
    num_epochs=1,
    label_name="Class").map(map_func)

In [12]:
batch = 0
for feature_batch, label_batch in pistachio_train_batches.take(2):
    # print(f'{batch}, {label_batch.shape}')
    # cat_batch = tf.stack([feature_batch['AREA'],feature_batch['PERIMETER']],axis=1)
    # cat_batch = tf.stack([feature_batch[k] for k in feature_batch],axis=1)

    # batch += 1
    
    print("'label': {}".format(label_batch))
    # print(cat_batch)
    print(f"features batch shape: {feature_batch.shape}")
    # print(feature_batch.shape)
    

'label': [[0]
 [1]
 [1]
 [0]]
features batch shape: (4, 16)
'label': [[0]
 [0]
 [1]
 [0]]
features batch shape: (4, 16)


2024-05-03 15:35:45.953447: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Model

In [47]:
# from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization
# from tensorflow.keras import Model

# class PistachioModel(Model):
#     def __init__(self, units: int=10):
#         super().__init__()
#         self._units = units
    

#     def build(self, input_shape):
#         self.bn = BatchNormalization(axis=0, input_shape=input_shape)
#         self.d1 = Dense(self._units, activation='relu', input_shape=input_shape)
#         self.d2 = Dense(self._units)
#         self.lout = Dense(1, activation='sigmoid')
        


#     def call(self, x):
#         x = self.bn(x)
#         x = self.d1(x)
#         x = self.d2(x)
#         return self.lout(x)

# # Create an instance of the model
# model = PistachioModel()

## Keras model.fit api

In [48]:
# model.compile(optimizer='adam',
#               loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
#               metrics=['accuracy', 'auc'])







In [8]:
# model.fit(pistachio_train_batches, epochs=10)

## sequential model

In [13]:
model2 = tf.keras.models.Sequential([
  tf.keras.layers.BatchNormalization(), 
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(16),
  tf.keras.layers.Dense(1, activation='sigmoid')
])


model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy', 'auc', 'precision', 'recall'])

In [14]:
model2.fit(pistachio_train_batches, epochs=20, validation_data=validation_data)


Epoch 1/20
    389/Unknown 2s 1ms/step - accuracy: 0.5741 - auc: 0.6996 - loss: 0.6420 - precision: 0.6184 - recall: 0.1590

2024-05-03 15:36:01.878182: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


398/398 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.5760 - auc: 0.7010 - loss: 0.6409 - precision: 0.6215 - recall: 0.1626 - val_accuracy: 0.7602 - val_auc: 0.8953 - val_loss: 0.5415 - val_precision: 0.8246 - val_recall: 0.5222
Epoch 2/20
 41/398 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6668 - auc: 0.8291 - loss: 0.5663 - precision: 0.8297 - recall: 0.3873  

2024-05-03 15:36:02.279921: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


398/398 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7368 - auc: 0.8232 - loss: 0.5400 - precision: 0.7838 - recall: 0.5596 - val_accuracy: 0.8869 - val_auc: 0.9188 - val_loss: 0.4298 - val_precision: 0.8824 - val_recall: 0.8333
Epoch 3/20


...

2024-05-03 15:36:20.340164: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-05-03 15:36:20.455190: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


398/398 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7860 - auc: 0.8650 - loss: 0.4524 - precision: 0.7654 - recall: 0.7332 - val_accuracy: 0.8733 - val_auc: 0.9321 - val_loss: 0.3566 - val_precision: 0.8229 - val_recall: 0.8778
Epoch 20/20


2024-05-03 15:36:21.554776: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-05-03 15:36:21.650230: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


398/398 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7822 - auc: 0.8420 - loss: 0.4856 - precision: 0.7627 - recall: 0.7279 - val_accuracy: 0.8597 - val_auc: 0.9328 - val_loss: 0.3545 - val_precision: 0.7980 - val_recall: 0.8778


2024-05-03 15:36:22.903813: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-05-03 15:36:23.020206: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


In [17]:
model2.evaluate(pistachio_test_data,verbose=2)


85/85 - 0s - 2ms/step - accuracy: 0.8698 - auc: 0.9361 - loss: 0.3439 - precision: 0.8252 - recall: 0.8613


2024-05-03 15:37:06.076313: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


[0.343885213136673,
 0.8698225021362305,
 0.9361041188240051,
 0.8251748085021973,
 0.8613138794898987]

In [18]:
for features, labels in pistachio_test_data.take(1):
    predictions = model2(features)
    for p,l in zip(predictions, labels):
        print(f'predicted prob: {p}, label: {l}')

predicted prob: [0.6096879], label: [1]
predicted prob: [0.13835986], label: [0]
predicted prob: [0.06268708], label: [0]
predicted prob: [0.10202026], label: [0]


2024-05-03 15:37:17.202003: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## custom training loop stuff

In [22]:
model3 = tf.keras.models.Sequential([
  tf.keras.layers.BatchNormalization(), 
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(16),
  tf.keras.layers.Dense(1, activation='sigmoid')
])



In [24]:
train_loss = []
valid_loss = []

optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4)
loss=tf.keras.losses.BinaryCrossentropy(from_logits=False)

epochs = 2
steps_per_output=20
for epoch in range(epochs):

    for batch_no, (train_x, train_y) in enumerate(pistachio_train_batches):
        with tf.GradientTape() as tape:
            output = model3(train_x)
            the_loss = loss(train_y, output)
        train_loss.append(the_loss)
            

        grads = tape.gradient(the_loss, model3.trainable_weights)
        optimizer.apply_gradients(zip(grads, model3.trainable_weights))

        if batch_no % steps_per_output == 0:
            print(f'epoch {epoch}, batch {batch_no}, training loss {the_loss}')
            
        



epoch 0, batch 0, training loss 14381.3994140625
epoch 0, batch 20, training loss 7997.92041015625
epoch 0, batch 40, training loss 2292.798828125
epoch 0, batch 60, training loss 8035.916015625
epoch 0, batch 80, training loss 3904.803466796875
epoch 0, batch 100, training loss 1452.4813232421875
epoch 0, batch 120, training loss 329.4244689941406
epoch 0, batch 140, training loss 330.1300048828125
epoch 0, batch 160, training loss 211.73709106445312
epoch 0, batch 180, training loss 247.14169311523438
epoch 0, batch 200, training loss 197.73309326171875
epoch 0, batch 220, training loss 0.0
epoch 0, batch 240, training loss 660.2647705078125
epoch 0, batch 260, training loss 364.07586669921875
epoch 0, batch 280, training loss 620.1875
epoch 0, batch 300, training loss 410.9370422363281
epoch 0, batch 320, training loss 89.31468963623047
epoch 0, batch 340, training loss 57.08540344238281
epoch 0, batch 360, training loss 287.11541748046875
epoch 0, batch 380, training loss 201.55319

2024-05-03 15:58:08.700327: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


epoch 1, batch 0, training loss 1578.214599609375
epoch 1, batch 20, training loss 245.0712890625
epoch 1, batch 40, training loss 169.11178588867188
epoch 1, batch 60, training loss 251.33184814453125
epoch 1, batch 80, training loss 859.609375
epoch 1, batch 100, training loss 320.0411071777344
epoch 1, batch 120, training loss 17.254487991333008
epoch 1, batch 140, training loss 163.32241821289062
epoch 1, batch 160, training loss 204.7286376953125
epoch 1, batch 180, training loss 88.8787841796875
epoch 1, batch 200, training loss 97.44146728515625
epoch 1, batch 220, training loss 938.4825439453125
epoch 1, batch 240, training loss 587.4556274414062
epoch 1, batch 260, training loss 194.38888549804688
epoch 1, batch 280, training loss 303.0118103027344
epoch 1, batch 300, training loss 501.0601806640625
epoch 1, batch 320, training loss 123.9653549194336
epoch 1, batch 340, training loss 86.04116821289062
epoch 1, batch 360, training loss 629.866455078125
epoch 1, batch 380, train

2024-05-03 15:58:29.213636: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch

In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [ ]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_state()
  train_accuracy.reset_state()
  test_loss.reset_state()
  test_accuracy.reset_state()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result():0.2f}, '
    f'Accuracy: {train_accuracy.result() * 100:0.2f}, '
    f'Test Loss: {test_loss.result():0.2f}, '
    f'Test Accuracy: {test_accuracy.result() * 100:0.2f}'
  )